# 04_Oracle - Sistema RAG para Prediccion de Muerte de Startups

## Que es este notebook? 

Construimos el **Oraculo de la Muerte**: un sistema RAG que predice como morira tu startup basandose en 409 startups que ya fracasaron.

## Arquitectura RAG

```
RETRIEVAL (buscar)     --> Usuario describe startup --> Buscar similares en base de datos
        ↓
AUGMENTED (contexto)   --> Crear prompt con info de startups muertas similares
        ↓
GENERATION (generar)   --> LLM genera prediccion personalizada y sarcastica
```

## Que aprenderemos? 

| Seccion | Contenido |
|---------|-----------|
| Setup | Instalar librerias, cargar datos |
| Metodo 1 | Busqueda manual con Cosine Similarity |
| Metodo 2 | Busqueda con ChromaDB (Vector DB) |
| Comparacion | Cuando usar cada metodo |

## Tecnologias

| Tecnologia | Para que |
|------------|----------|
| Sentence Transformers | Generar embeddings |
| Cosine Similarity | Busqueda manual |
| ChromaDB | Base de datos vectorial |
| Gemini 2.0 Flash | Generar predicciones |

# 
# SETUP: Instalacion y Carga de Datos
# 

In [0]:
%pip install sentence-transformers google-generativeai chromadb -q

print("Dependencias instaladas!   ✓")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Dependencias instaladas!   ✓


In [0]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Rutas
gold_path = "/Workspace/startup-death-oracle/data/gold/"

# Cargar embeddings y datos
embeddings = np.load(gold_path + "embeddings.npy")
df = pd.read_csv(gold_path + "failures_with_embeddings.csv")

# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

print("DATOS CARGADOS:")
print("=" * 50)
print(f"Startups: {len(df)}")
print(f"Embeddings: {embeddings.shape}")
print(f"Modelo: all-MiniLM-L6-v2  ✓")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

DATOS CARGADOS:
Startups: 409
Embeddings: (409, 384)
Modelo: all-MiniLM-L6-v2  ✓


In [0]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Cargar variables del archivo .env
load_dotenv()

# Obtener API key
GEMINI_API_KEY = os. getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("❌ Falta GEMINI_API_KEY.  Crea archivo . env")

genai.configure(api_key=GEMINI_API_KEY)
print("Gemini configurado!   ✓")

Gemini configurado!   ✓


## Que cargamos?

| Variable | Tipo | Contenido |
|----------|------|-----------|
| `df` | DataFrame | 409 startups muertas con info |
| `embeddings` | numpy array | Matriz 409 x 384 (vectores) |
| `model` | SentenceTransformer | Modelo para crear nuevos embeddings |
| `llm` | GenerativeModel | Gemini para generar predicciones |

## Archivos usados

| Archivo | Descripcion |
|---------|-------------|
| `embeddings.npy` | Vectores pre-calculados |
| `failures_with_embeddings.csv` | Datos + embeddings en texto |

## Modelo de embeddings

Usamos **all-MiniLM-L6-v2** porque:
- Rapido y ligero
- Vectores de 384 dimensiones
- Buena calidad para textos cortos

# 
# METODO 1: Busqueda Manual con Cosine Similarity
# 

## Que es? 

Busqueda de startups similares calculando similitud coseno **manualmente**.

## Flujo

```
Usuario escribe descripcion
        ↓
Convertir a embedding (384 dimensiones)
        ↓
Comparar con TODOS los 409 embeddings
        ↓
Ordenar por similitud
        ↓
Devolver top 3
```

## Ventajas

| Pro | Descripcion |
|-----|-------------|
| Transparente | Ves exactamente como funciona |
| Sin dependencias | Solo sklearn |
| Educativo | Perfecto para aprender |

## Desventajas

| Contra | Descripcion |
|--------|-------------|
| Lento | Compara con TODOS los documentos |
| No escala | Millones de docs = muy lento |
| No persiste | Hay que cargar CSV cada vez |

## Cuando usarlo? 

- Datasets pequeños (< 10,000 docs)
- Prototipos y MVPs
- Cuando quieres entender como funciona

Funcion buscar_startups_similares

In [0]:
def buscar_startups_similares(descripcion_usuario, top_k=3):
    """
    Busca las startups fallidas mas similares a la descripcion del usuario.
    
    Args:
        descripcion_usuario: texto describiendo la startup
        top_k: cantidad de resultados a retornar
    
    Returns:
        DataFrame con las startups mas similares
    """
    
    # 1. Convertir descripcion a embedding
    embedding_usuario = model.encode([descripcion_usuario])
    
    # 2. Calcular similitud con todas las startups
    similitudes = cosine_similarity(embedding_usuario, embeddings)[0]
    
    # 3. Obtener indices de las top_k mas similares
    top_indices = similitudes.argsort()[-top_k:][::-1]
    
    # 4. Crear dataframe con resultados
    resultados = df.iloc[top_indices]. copy()
    resultados['similitud'] = similitudes[top_indices]
    
    return resultados[['name', 'sector', 'text', 'similitud']]

# Probar
resultado = buscar_startups_similares("food delivery app with high costs")
print("PRUEBA DE BUSQUEDA:")
print("=" * 50)
print(resultado. to_string())

PRUEBA DE BUSQUEDA:
      name                           sector                                                                                                                                                                                                                                              text  similitud
54   Sprig                      Health Care                                                            company sprig. sector health care. what they did healthy meal delivery. why they failed high costs and competition. lesson need cost advantage. causes: giants, no_budget, competition   0.409831
62  Chef'd  Accommodation and Food Services               company chefd. sector accommodation and food services. what they did meal kit delivery. why they failed closed 2019 high costs lost to blue apron. lesson costs cook meal kits. causes: giants, competition, high_operational_costs   0.406394
76  Ritual  Accommodation and Food Services  company ritual. sector accommodation and foo

Funcion preparar_contexto

In [0]:
def preparar_contexto(resultados):
    """
    Convierte los resultados de busqueda en texto para el LLM. 
    """
    contexto = "STARTUPS SIMILARES QUE MURIERON:\n\n"
    
    for i, row in resultados.iterrows():
        contexto += f"- {row['name']} ({row['sector']})\n"
        contexto += f"  {row['text']}\n\n"
    
    return contexto

# Probar
contexto = preparar_contexto(resultado)
print(contexto)

STARTUPS SIMILARES QUE MURIERON:

- Sprig (Health Care)
  company sprig. sector health care. what they did healthy meal delivery. why they failed high costs and competition. lesson need cost advantage. causes: giants, no_budget, competition

- Chef'd (Accommodation and Food Services)
  company chefd. sector accommodation and food services. what they did meal kit delivery. why they failed closed 2019 high costs lost to blue apron. lesson costs cook meal kits. causes: giants, competition, high_operational_costs

- Ritual (Accommodation and Food Services)
  company ritual. sector accommodation and food services. what they did food pickup app. why they failed faded 2022 lost to doordash pickup waned. lesson delivery beats pickup. causes: giants, competition, trend_shifts, high_operational_costs




Funcion generar_prediccion v1

In [0]:
def generar_prediccion(descripcion_usuario):
    """
    Genera prediccion de muerte usando RAG con busqueda manual. 
    """
    
    # 1.  Buscar startups similares (Retrieval)
    resultados = buscar_startups_similares(descripcion_usuario)
    
    # 2.  Preparar contexto (Augmented)
    contexto = preparar_contexto(resultados)
    
    # 3. Crear prompt
    prompt = f"""
Eres el RAG de la Muerte de Startups.  Predices como morira una startup basandote en startups similares que ya murieron. 

DESCRIPCION DE LA STARTUP:
{descripcion_usuario}

{contexto}

Genera una prediccion UNICA y CREATIVA con:
1. Probabilidad de muerte (0-100%)
2.  Causa mas probable (se especifico, no generico)
3.  Tiempo estimado antes de morir
4. Epitafio gracioso para su tumba (1 frase)
5. Ultima voluntad (que deberian hacer antes de morir)

Se sarcastico, creativo y NO repitas consejos genericos como "busca un nicho". 
Cada prediccion debe ser diferente y personalizada.
"""
    
    # 4. Generar respuesta (Generation)
    response = llm.generate_content(prompt)
    
    return response.text

# Probar
print("PREDICCION METODO 1:")
print("=" * 50)
prediccion = generar_prediccion("the startup is called FastyFood is a food delivery app competing with uber eats")
print(prediccion)

PREDICCION METODO 1:
¡Aquí está la predicción RAG para FastyFood, adornada con un toque de sarcasmo y creatividad!

**Predicción de Muerte de FastyFood:**

1.  **Probabilidad de Muerte:** 85%

2.  **Causa Más Probable:** "La Uber-Comida Apocalíptica". FastyFood será víctima de la implacable estrategia de "precios predatorios" de Uber Eats. Uber Eats, cual dragón hambriento, quemará efectivo hasta que FastyFood no pueda competir con sus promociones y descuentos absurdos. No importa cuán rápido sea FastyFood, no podrá superar el pozo sin fondo de dinero de su competidor. Se estancarán intentando justificar precios más altos con un "servicio más personalizado", mientras los usuarios (con razón) eligen la comida más barata.

3.  **Tiempo Estimado Antes de Morir:** 18 meses. La lenta agonía se extenderá a medida que los fondos iniciales se agoten y la desesperación se instale.

4.  **Epitafio Gracioso:** "Aquí yace FastyFood. Demasiado lento, demasiado tarde, demasiado 'cool' para sobrevivi

## Diccionario de Causas de Muerte

| Codigo | Significado | Ejemplo |
|--------|-------------|---------|
| `giants` | Competencia con gigantes | Uber, Google, Amazon aplastaron el negocio |
| `no_budget` | Se quedaron sin dinero | No consiguieron mas funding |
| `competition` | Competencia general | Demasiados competidores en el mercado |
| `high_operational_costs` | Costos operativos altos | Gastaban mas de lo que ganaban |
| `trend_shifts` | El mercado cambio | La tendencia paso de moda |
| `no_product_market_fit` | Sin product-market fit | Nadie queria el producto |
| `bad_timing` | Mal timing | Lanzaron muy temprano o muy tarde |
| `legal_issues` | Problemas legales | Regulaciones, demandas |
| `team_issues` | Problemas de equipo | Conflictos entre fundadores |
| `pivot_failed` | Pivot fallido | Intentaron cambiar y no funciono |


# METODO 2: Busqueda con ChromaDB (Vector Database)

## Que es? 

**ChromaDB** es una base de datos vectorial optimizada para busqueda semantica. 

## Flujo

```
Usuario escribe descripcion
        ↓
Convertir a embedding
        ↓
ChromaDB busca (optimizado internamente)
        ↓
Devolver top 3
```

## Ventajas sobre Metodo 1

| Metrica | Cosine Manual | ChromaDB |
|---------|---------------|----------|
| Lineas de codigo | 10 | 1 |
| 409 docs | ~50ms | ~10ms |
| 10,000 docs | ~500ms | ~15ms |
| 1,000,000 docs | ~50s | ~50ms |

## Caracteristicas

| Feature | Descripcion |
|---------|-------------|
| Metadatos | Guarda info extra (nombre, sector) |
| Persistencia | Puede guardar a disco |
| Indices | Busqueda optimizada |

## Cuando usarlo?

- Produccion
- Datasets grandes (> 10,000 docs)
- Cuando necesitas velocidad y escalabilidad

Crear ChromaDB

In [0]:
import chromadb

# Crear cliente (en memoria)
chroma_client = chromadb.Client()

# Crear o obtener coleccion
collection = chroma_client. get_or_create_collection(
    name="startups_muertas",
    metadata={"description": "Startups que fracasaron"}
)

# Solo agregar si esta vacia
if collection.count() == 0:
    # Cargar datos del Gold
    df_gold = pd. read_csv("/Workspace/startup-death-oracle/data/gold/failures_with_embeddings.csv")

    # Agregar a ChromaDB
    collection.add(
        ids=[str(i) for i in range(len(df_gold))],
        documents=df_gold['text'].tolist(),
        metadatas=[{"name": row['name'], "sector": row['sector']} for _, row in df_gold.iterrows()],
        embeddings=df_gold['embedding']. apply(eval).tolist()
    )
    print(f"ChromaDB creado con {collection.count()} startups  ✓")
else:
    print(f"ChromaDB ya tiene {collection.count()} startups  ✓")

ChromaDB creado con 409 startups  ✓


Funcion buscar_con_chroma

In [0]:
def buscar_con_chroma(descripcion_usuario, n_resultados=3):
    """
    Busca startups similares usando ChromaDB.
    Mucho mas simple que cosine_similarity manual! 
    """
    
    # Convertir texto a embedding
    embedding_usuario = model. encode(descripcion_usuario). tolist()
    
    # Buscar en ChromaDB (1 linea!)
    resultados = collection.query(
        query_embeddings=[embedding_usuario],
        n_results=n_resultados
    )
    
    return resultados

# Probar
resultados = buscar_con_chroma("food delivery app")
print("RESULTADOS CHROMADB:")
print("=" * 50)
for i, (name, doc) in enumerate(zip(resultados['metadatas'][0], resultados['documents'][0])):
    print(f"{i+1}. {name['name']}: {doc[:80]}...")

RESULTADOS CHROMADB:
1. Ritual: company ritual. sector accommodation and food services. what they did food picku...
2. Foodler: company foodler. sector accommodation and food services. what they did food deli...
3. OrderAhead: company orderahead. sector accommodation and food services. what they did restau...


Funcion generar_prediccion_v2 (con ChromaDB)

In [0]:
def generar_prediccion_v2(descripcion_usuario):
    """
    Version mejorada usando ChromaDB. 
    """
    
    # 1. Buscar con ChromaDB (Retrieval)
    resultados = buscar_con_chroma(descripcion_usuario, n_resultados=3)
    
    # 2.  Preparar contexto (Augmented)
    documentos = resultados['documents'][0]
    metadatas = resultados['metadatas'][0]
    
    contexto = "STARTUPS SIMILARES QUE MURIERON:\n\n"
    for i, (doc, meta) in enumerate(zip(documentos, metadatas), 1):
        contexto += f"{i}. {meta['name']} ({meta['sector']})\n"
        contexto += f"   {doc}\n\n"
    
    # 3. Crear prompt
    prompt = f"""
Eres el RAG de la Muerte de Startups.  Predices como morira una startup basandote en startups similares que ya murieron. 

DESCRIPCION DE LA STARTUP:
{descripcion_usuario}

{contexto}

Genera una prediccion UNICA y CREATIVA con:
1.  Probabilidad de muerte (0-100%)
2. Causa mas probable (se especifico, no generico)
3.  Tiempo estimado antes de morir
4.  Epitafio gracioso para su tumba (1 frase)
5.  Ultima voluntad (que deberian hacer antes de morir)

Se sarcastico, creativo y NO repitas consejos genericos como "busca un nicho".
Cada prediccion debe ser diferente y personalizada. 
"""
    
    # 4.  Generar respuesta (Generation)
    response = llm. generate_content(prompt)
    
    return response.text

# Probar
print("PREDICCION METODO 2 (ChromaDB):")
print("=" * 50)
prediccion = generar_prediccion_v2("startup is called fastfood, food delivery app competing with uber eats and glovo")
print(prediccion)

PREDICCION METODO 2 (ChromaDB):
¡Adelante con la guadaña! Analicemos el destino de FastFood:

**Predicción R.I.P. FastFood**

1.  **Probabilidad de Muerte:** 75%

2.  **Causa más probable:** "La Salsa Secreta Demasiado Secreta". FastFood, en un intento desesperado por diferenciarse, crea una "salsa secreta" que es tan secreta que ni ellos saben qué demonios lleva. Los clientes la prueban, quedan confundidos y prefieren volver a la familiaridad de Uber Eats y Glovo, donde al menos saben que la mayonesa es mayonesa. La incapacidad de articular una propuesta de valor clara y atractiva (más allá de la misteriosa salsa) los hunde.

3.  **Tiempo estimado antes de morir:** 18 meses. Suficiente para quemar la inversión inicial intentando explicarle al mundo qué rayos es esa salsa.

4.  **Epitafio:** "Aquí yace FastFood. Al menos la salsa secreta... nunca se reveló."

5.  **Última Voluntad:** Antes de cerrar, deberían organizar una cata a ciegas de la "salsa secreta" donde los clientes voten po

#
# COMPARACION: Manual vs ChromaDB
#

## Resumen de Funciones

| Metodo | Busqueda | Prediccion |
|--------|----------|------------|
| Manual | `buscar_startups_similares()` | `generar_prediccion()` |
| ChromaDB | `buscar_con_chroma()` | `generar_prediccion_v2()` |

## Comparacion Detallada

| Aspecto | Metodo 1 (Manual) | Metodo 2 (ChromaDB) |
|---------|-------------------|---------------------|
| Complejidad codigo | Media (10 lineas) | Baja (1 linea) |
| Velocidad | O(n) - compara con todos | O(log n) - indexado |
| Escalabilidad | Baja | Alta |
| Dependencias | sklearn | chromadb |
| Persistencia | No | Si (opcional) |
| Metadatos | Manual | Integrado |

## Cual usar? 

```
¿Estas aprendiendo RAG?
        ↓
   SI → Metodo 1 (ves todo el proceso)
   NO → Metodo 2 (mas simple y rapido)

¿Vas a produccion?
        ↓
   SI → Metodo 2 (escala mejor)
   NO → Cualquiera funciona
```

## Siguiente Paso

Crear app con **Streamlit** usando `generar_prediccion_v2()` para que cualquier usuario pueda consultar al Oraculo. 